In [1]:
#데이터 핸들링 라이브러리
import pandas as pd
import numpy as np
#데이터 시각화 라이브러리
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcParams.update({'font.family':'AppleGothic'})
mpl.rc('axes', unicode_minus=False)
%config InlineBackend.figure_format = 'retina'


#웹 크롤링에 사용될 라이브러리
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from time import sleep
from pprint import pprint

#기타
import re
import json
from geopy.geocoders import Nominatim

In [22]:
#교통사고 데이터 불러오기
import sys
sys.path.append('/Users/wooseongkyun/코드_아카이브/프로젝트2_머신러닝과EDA/binomial_classification/preprocessing_data.py')
from preprocessing_data import f1_read

df= f1_read()
df

데이터 불러오기 및 통합에 걸린 시간은 61.573285818099976 입니다


,사고번호,사고일시,요일,시군구,사고내용,사망자수,중상자수,경상자수,부상신고자수,사고유형,...,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도
0,2020010100100011,2020년 1월 1일 00시,수요일,대구광역시 달서구 죽전동,중상사고,0,1,0,0,차대차 - 측면충돌,...,맑음,단일로 - 기타,승용,남,70세,상해없음,이륜,남,18세,중상
1,2020010100100048,2020년 1월 1일 02시,수요일,대구광역시 달서구 송현동,경상사고,0,0,1,0,차대차 - 측면충돌,...,맑음,교차로 - 교차로안,원동기,남,50세,상해없음,승용,남,26세,경상
2,2020010100100071,2020년 1월 1일 05시,수요일,대구광역시 북구 대현동,경상사고,0,0,1,0,차대차 - 기타,...,맑음,교차로 - 교차로부근,기타불명,기타불명,미분류,기타불명,승용,남,37세,경상
3,2020010100100072,2020년 1월 1일 05시,수요일,대구광역시 중구 삼덕동1가,경상사고,0,0,1,0,차대사람 - 길가장자리구역통행중,...,맑음,단일로 - 기타,승용,남,18세,상해없음,보행자,남,21세,경상
4,2020010100100087,2020년 1월 1일 06시,수요일,대구광역시 수성구 두산동,경상사고,0,0,1,0,차대차 - 기타,...,맑음,단일로 - 기타,승용,남,45세,상해없음,승용,남,26세,경상
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412779,2021123100100475,2021년 12월 31일 18시,금요일,울산광역시 북구 호계동,중상사고,0,1,0,0,차대사람 - 횡단중,...,맑음,교차로 - 교차로횡단보도내,승용,남,33세,상해없음,보행자,남,65세,중상
412780,2021123100100476,2021년 12월 31일 18시,금요일,울산광역시 북구 중산동,경상사고,0,0,1,0,차대차 - 추돌,...,맑음,단일로 - 기타,승용,남,38세,상해없음,승용,남,32세,경상
412781,2021123100100477,2021년 12월 31일 18시,금요일,울산광역시 북구 진장동,경상사고,0,0,1,0,차대차 - 측면충돌,...,맑음,단일로 - 기타,승용,남,71세,상해없음,승용,여,30세,경상
412782,2021123100100593,2021년 12월 31일 21시,금요일,울산광역시 남구 야음동,경상사고,0,0,1,0,차대차 - 후진중충돌,...,맑음,단일로 - 기타,승용,남,28세,상해없음,이륜,남,17세,경상


In [54]:
location_lst=df['시군구'].tolist()
Sido_lst=[]
for location in location_lst:
     splited_loc= location.split()
     Sido_lst.append(splited_loc[0])

Sido_lst
df['시도']=Sido_lst

year_lst=[]
date_lst=df['사고일시']
for date in date_lst:
    splited_date= date.split()
    year_lst.append(splited_date[0])

year_lst
df['년도']=year_lst

In [55]:
#사용되는 컬럼 정보만 가져오기
df= df[['사고일시','시도','기상상태','사고내용','년도']]

,사고번호,사고일시,요일,시군구,사고내용,사망자수,중상자수,경상자수,부상신고자수,사고유형,...,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,시도,년도
0,2020010100100011,2020년 1월 1일 00시,수요일,대구광역시 달서구 죽전동,중상사고,0,1,0,0,차대차 - 측면충돌,...,승용,남,70세,상해없음,이륜,남,18세,중상,대구광역시,2020년
1,2020010100100048,2020년 1월 1일 02시,수요일,대구광역시 달서구 송현동,경상사고,0,0,1,0,차대차 - 측면충돌,...,원동기,남,50세,상해없음,승용,남,26세,경상,대구광역시,2020년
2,2020010100100071,2020년 1월 1일 05시,수요일,대구광역시 북구 대현동,경상사고,0,0,1,0,차대차 - 기타,...,기타불명,기타불명,미분류,기타불명,승용,남,37세,경상,대구광역시,2020년
3,2020010100100072,2020년 1월 1일 05시,수요일,대구광역시 중구 삼덕동1가,경상사고,0,0,1,0,차대사람 - 길가장자리구역통행중,...,승용,남,18세,상해없음,보행자,남,21세,경상,대구광역시,2020년
4,2020010100100087,2020년 1월 1일 06시,수요일,대구광역시 수성구 두산동,경상사고,0,0,1,0,차대차 - 기타,...,승용,남,45세,상해없음,승용,남,26세,경상,대구광역시,2020년
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412779,2021123100100475,2021년 12월 31일 18시,금요일,울산광역시 북구 호계동,중상사고,0,1,0,0,차대사람 - 횡단중,...,승용,남,33세,상해없음,보행자,남,65세,중상,울산광역시,2021년
412780,2021123100100476,2021년 12월 31일 18시,금요일,울산광역시 북구 중산동,경상사고,0,0,1,0,차대차 - 추돌,...,승용,남,38세,상해없음,승용,남,32세,경상,울산광역시,2021년
412781,2021123100100477,2021년 12월 31일 18시,금요일,울산광역시 북구 진장동,경상사고,0,0,1,0,차대차 - 측면충돌,...,승용,남,71세,상해없음,승용,여,30세,경상,울산광역시,2021년
412782,2021123100100593,2021년 12월 31일 21시,금요일,울산광역시 남구 야음동,경상사고,0,0,1,0,차대차 - 후진중충돌,...,승용,남,28세,상해없음,이륜,남,17세,경상,울산광역시,2021년


In [ ]:
#데이터 전처리하기
from preprocessing_data import f5_encdoding_OneHot
df= f5_encdoding_OneHot(df,['사고'])